[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Xbmovnn842WdkFmFIYfOpipn5wci49sP#scrollTo=C93b1yUGCMw5)

In [ ]:
%cd Documents/GitHub/Frame-level-student-engagement

C:\Users\rijju\Documents\GitHub\Frame-level-student-engagement


In [37]:
import pandas as pd
from DataFormatter import DataFormatter
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from Tab_CNN import CNNModel

In [27]:
!pwd

/c/Users/rijju/Documents/GitHub/Frame-level-student-engagement


In [ ]:
#replace the path according to the path to the dataset
df = pd.read_csv("Data/WACV_train_data.csv")
df= df.loc[:,"gaze_0_x":]
train_split, val_split = 0.7, 0.15

formatter = DataFormatter(train_split, val_split)
x_train, x_val, x_test, y_train, y_val, y_test = formatter.dataframeFormat(df)
# Instantiate the CNNModel class
num_classes = 3
inp_shape = x_train.shape[1:]
cnn_model = CNNModel(num_classes,inp_shape)

# Train the model
batch_size = 32
epochs = 1
history = cnn_model.train(x_train, y_train, batch_size, epochs)

# Evaluate the model
accuracy = cnn_model.evaluate(x_test, y_test)
print("Test accuracy:", accuracy)

model= cnn_model.model
# make a reference to model's input layer
inp = model.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(4, activation='softmax')

# connect our new layer to the second to last layer in model, and make a reference to it
out = new_classification_layer(model.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

dfD = pd.read_csv("Data/DAiSEE_TL_data.csv")
dfD= dfD.loc[:,"gaze_0_x":]
xD_train, xD_val, xD_test, yD_train, yD_val, yD_test = formatter.dataframeFormat(dfD)

history2 = model_new.fit(xD_train, yD_train, 
                         batch_size=batch_size, 
                         epochs=epochs, 
                         validation_data=(xD_val, yD_val))

#comparing both model's loss and accuracy
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["loss"], label="WACV val loss")
ax.plot(history2.history["loss"], label="DAiSEE val loss")
# ax.set_title("validation loss")
ax.legend(loc="upper left")
ax.set_xlabel("epochs")
ax.set_ylabel("validation loss")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["accuracy"], label="WACV val accuracy")
ax2.plot(history2.history["accuracy"],label="DAiSEE val accuracy")
# ax2.set_title("validation accuracy")
ax2.legend(loc="lower right")
ax2.set_xlabel("epochs")
ax2.set_ylabel("validation accuracy")
ax2.set_ylim(0, 1)
plt.savefig("Results/TF_los_acc_OF.pdf")
plt.show()

In [ ]:
yD_pred = model_new.predict(xD_test)

# Reshape predicted and true labels into binary vectors
y_pred_binary = np.argmax(yD_pred, axis=1)
y_true_binary = np.argmax(yD_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_true_binary, y_pred_binary)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

In [ ]:
#save the model

# serialize model to YAML
model_yaml = model_new.to_json()
with open("Model/model_OF.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model_new.save_weights("Model/model_OF.h5")
print("Saved model to disk")

In [ ]:
#load the model
from tensorflow.keras.models import Sequential, model_from_json
yaml_file = open('Model_OF.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_json(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("Model/model_OF.h5")
print("Loaded model from disk")